In [1]:
import os
import csv
from glob import glob

import cv2
import numpy as np

In [2]:
input_dir = "./data/train"
output_dir = "./data/train_augmentations"
train_images = glob(input_dir + "/*/*")

In [3]:
# Apply transformations to all train data and create data_augmented folder
# Note: This process may take a long time depending on the situation.

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def elastic_distortion(image, alpha=20, sigma=5):
    """
    이미지에 Elastic Distortion을 적용하는 함수.
    
    Args:
        image: 입력 이미지 (numpy array)
        alpha: 왜곡 강도 (픽셀 이동 크기)
        sigma: 가우시안 필터 표준 편차 (왜곡의 매끄러움)
    
    Returns:
        변형된 이미지 (numpy array)
    """
    random_state = np.random.RandomState(None)

    shape = image.shape
    dx = random_state.rand(*shape) * 2 - 1  # 랜덤한 변위 필드
    dy = random_state.rand(*shape) * 2 - 1  # 랜덤한 변위 필드

    # 가우시안 필터 적용
    dx = cv2.GaussianBlur(dx, (17, 17), sigma)
    dy = cv2.GaussianBlur(dy, (17, 17), sigma)

    # 변위 필드를 alpha 값에 맞게 조정
    dx = dx * alpha
    dy = dy * alpha

    # 원본 이미지 좌표 생성
    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))

    # 새로운 좌표로 변형
    map_x = (x + dx).astype(np.float32)
    map_y = (y + dy).astype(np.float32)

    # remap을 사용해 이미지를 변형된 좌표로 매핑
    distorted_image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    return distorted_image

def apply_transformations(img) -> list:
    """
    Apply Gaussian Blur, Canny, Morphology Open, and Morphology Close transformations to the input image.
    
    Parameters:
    img (numpy.ndarray): Input grayscale image.
    
    Returns:
    list: List containing transformed images [Gaussian Blur, Canny, Morphology Open, Morphology Close].
    """
    # Apply Gaussian Blur
    gaussian_blur = cv2.GaussianBlur(img, (5, 5), 0)
    
    # Apply Canny edge detection
    canny = cv2.Canny(img, 100, 200)
    
    # Define kernel for morphological operations
    kernel = np.ones((5, 5), np.uint8)
    
    # Apply Morphology Open (Erosion followed by Dilation)
    # morph_open = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    
    # Apply Morphology Close (Dilation followed by Erosion)
    morph_close = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    
    return [gaussian_blur, canny, morph_close]

# Make train.csv dictionary
label_map = {}
with open('./data/train.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # row[1]은 이미지 경로, row[2]는 클래스 레이블
        image_name = row[1]
        class_label = row[2]
        label_map[image_name] = class_label

# transform from all train images to augmentation images 
with open('./data/train_augmentations.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['image_path', 'target'])  # 헤더 작성       

    # transform from all train images to augmentation images 
    for i, img_path in enumerate(train_images):
        # Load the image in grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        print(i, img_path)
        if img is None:
            continue
        
        # Apply transformations
        transformed_img_list = apply_transformations(img)

        # 이미지 경로에서 파일 이름 추출 (예: n02106550/sketch_41.JPEG)
        relative_img_path = os.path.relpath(img_path, './data/train')
        class_label = label_map.get(relative_img_path)

        # Generate output file names
        sequential_number = i + 1
        base_name = f'sketch_{sequential_number}'

        # Save the original image in the same output directory
        original_output_path = os.path.join(output_dir, f'{base_name}_original.JPEG')
        cv2.imwrite(original_output_path, img)

        # Save each transformed image with a specific name
        transformations = ['gaussian_blur', 'canny', 'morph_close']
        for j, transformed_img in enumerate(transformed_img_list):
            transformed_output_path = os.path.join(output_dir, f'{base_name}_{transformations[j]}.JPEG')
            cv2.imwrite(transformed_output_path, transformed_img)
            
            # 이미지의 상대 경로를 얻어 저장
            transformed_relative_path = os.path.relpath(transformed_output_path, './data')
            
            # CSV 파일에 추가 (상대 경로로 저장)
            csv_writer.writerow([transformed_relative_path, class_label])

        # Save the original image path to CSV
        original_relative_path = os.path.relpath(original_output_path, './data')
        csv_writer.writerow([original_relative_path, class_label])

    print("Image augmentation completed and saved to", output_dir)



0 ./data/train/n02088238/sketch_42.JPEG
1 ./data/train/n02088238/sketch_19.JPEG
2 ./data/train/n02088238/sketch_16.JPEG
3 ./data/train/n02088238/sketch_13.JPEG
4 ./data/train/n02088238/sketch_41.JPEG
5 ./data/train/n02088238/sketch_31.JPEG
6 ./data/train/n02088238/sketch_38.JPEG
7 ./data/train/n02088238/sketch_32.JPEG
8 ./data/train/n02088238/sketch_39.JPEG
9 ./data/train/n02088238/sketch_37.JPEG
10 ./data/train/n02088238/sketch_28.JPEG
11 ./data/train/n02088238/sketch_47.JPEG
12 ./data/train/n02088238/sketch_10.JPEG
13 ./data/train/n02088238/sketch_15.JPEG
14 ./data/train/n02088238/sketch_17.JPEG
15 ./data/train/n02088238/sketch_18.JPEG
16 ./data/train/n02088238/sketch_1.JPEG
17 ./data/train/n02088238/sketch_14.JPEG
18 ./data/train/n02088238/sketch_11.JPEG
19 ./data/train/n02088238/sketch_46.JPEG
20 ./data/train/n02088238/sketch_12.JPEG
21 ./data/train/n02088238/sketch_27.JPEG
22 ./data/train/n02088238/sketch_8.JPEG
23 ./data/train/n02088238/sketch_35.JPEG
24 ./data/train/n02088238/sk

In [4]:
import pandas as pd

csv_path = './data/train.csv'

# CSV 파일 읽기
# 첫 번쨰 열 제거
df = pd.read_csv(csv_path, usecols=[1, 2])
df

,class_name,image_path
0,n01872401,n01872401/sketch_50.JPEG
1,n02417914,n02417914/sketch_11.JPEG
2,n02106166,n02106166/sketch_3.JPEG
3,n04235860,n04235860/sketch_2.JPEG
4,n02056570,n02056570/sketch_40.JPEG
...,...,...
12774,n02107683,n02107683/sketch_7.JPEG
12775,n02108089,n02108089/sketch_32.JPEG
12776,n02129604,n02129604/sketch_7.JPEG
12777,n02325366,n02325366/sketch_46.JPEG


In [5]:
# Make train.csv dictionary
df_train = pd.read_csv('./data/train.csv', usecols=[1, 2, 3])
df_train


,class_name,image_path,target
0,n01872401,n01872401/sketch_50.JPEG,59
1,n02417914,n02417914/sketch_11.JPEG,202
2,n02106166,n02106166/sketch_3.JPEG,138
3,n04235860,n04235860/sketch_2.JPEG,382
4,n02056570,n02056570/sketch_40.JPEG,80
...,...,...,...
12774,n02107683,n02107683/sketch_7.JPEG,142
12775,n02108089,n02108089/sketch_32.JPEG,143
12776,n02129604,n02129604/sketch_7.JPEG,172
12777,n02325366,n02325366/sketch_46.JPEG,186


In [7]:
# CSV 파일만 작성되도록 수정한 코드
with open('./data/train_augmentations.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['image_path', 'target'])  # 헤더 작성       

    # 이미지 변환 및 저장 없이 경로와 레이블만 기록
    for i, img_path in enumerate(train_images):
        print(i, img_path)

        # 클래스 레이블 찾기
        class_label = df_train[df_train['image_path'] == img_path.removeprefix('./data/train/')]['target'].values[0]

        # Generate output file names (원본 및 변환된 이미지 이름 생성)
        sequential_number = i + 1
        base_name = f'sketch_{sequential_number}'

        # 변환된 이미지 파일명만 생성하여 CSV에 기록 (실제 변환 및 저장 없이)
        transformations = ['gaussian_blur', 'canny', 'morph_close']
        for j in range(len(transformations)):
            transformed_output_path = os.path.join(output_dir, f'{base_name}_{transformations[j]}.JPEG')

            # 이미지의 상대 경로를 얻어 저장 (이미지 저장 없이 경로만 기록)
            transformed_relative_path = os.path.relpath(transformed_output_path, './data/train_augmentations')
            
            # CSV 파일에 추가 (경로와 레이블 저장)
            csv_writer.writerow([transformed_relative_path, class_label])

        # 원본 이미지 경로도 CSV에 기록
        original_output_path = os.path.join(output_dir, f'{base_name}_original.JPEG')
        original_relative_path = os.path.relpath(original_output_path, './data/train_augmentations')
        csv_writer.writerow([original_relative_path, class_label])

    print("Image paths recorded in CSV without transformations and saving images.")

0 ./data/train/n02088238/sketch_42.JPEG
1 ./data/train/n02088238/sketch_19.JPEG
2 ./data/train/n02088238/sketch_16.JPEG
3 ./data/train/n02088238/sketch_13.JPEG
4 ./data/train/n02088238/sketch_41.JPEG
5 ./data/train/n02088238/sketch_31.JPEG
6 ./data/train/n02088238/sketch_38.JPEG
7 ./data/train/n02088238/sketch_32.JPEG
8 ./data/train/n02088238/sketch_39.JPEG
9 ./data/train/n02088238/sketch_37.JPEG
10 ./data/train/n02088238/sketch_28.JPEG
11 ./data/train/n02088238/sketch_47.JPEG
12 ./data/train/n02088238/sketch_10.JPEG
13 ./data/train/n02088238/sketch_15.JPEG
14 ./data/train/n02088238/sketch_17.JPEG
15 ./data/train/n02088238/sketch_18.JPEG
16 ./data/train/n02088238/sketch_1.JPEG
17 ./data/train/n02088238/sketch_14.JPEG
18 ./data/train/n02088238/sketch_11.JPEG
19 ./data/train/n02088238/sketch_46.JPEG
20 ./data/train/n02088238/sketch_12.JPEG
21 ./data/train/n02088238/sketch_27.JPEG
22 ./data/train/n02088238/sketch_8.JPEG
23 ./data/train/n02088238/sketch_35.JPEG
24 ./data/train/n02088238/sk